# Base Case Test with Known Result

In this notebook, we will demonstrate a base case test where we have a known result. This is a common practice in testing algorithms or functions to ensure they are working correctly.

In [ ]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

In [ ]:

# Install solver modules:
$ python -m amplpy.modules install highs gurobi

# Activate your license (e.g., free ampl.com/ce or ampl.com/courses licenses):
$ python -m amplpy.modules activate 887bd036-867c-4eb1-bb85-e70c449187de

# Import and instantiate AMPL object in Python:
from amplpy import AMPL
ampl = AMPL() # instantiate AMPL object

In [ ]:
shell "amplkey activate --uuid 887bd036-867c-4eb1-bb85-e70c449187de";

## I. Problem formulation

Consider a set ${S}$ of available stock materials that can be cut to size to produce a set of finished parts. Each stock $s\in S$ is characterized by a width $w^S_s$. A customer order is received to product a set of finished products $F$. Each finished product $f\in F$ is specified by a required number $d_f$ and width $w^F_f$. The base case test involves providing the algorithm or function with the simplest input or scenario for which we know the expected outcome. This allows us to verify if the algorithm or function produces the expected result under these conditions.

The **cutting stock problem** is to find a minimum lost function to fulfill the customer order from the stock materials.

In [ ]:
## PARAMETER
BOUND = 0.5 # over cut - maxi bound on fc quantity
MIN_MARGIN = 8

# TEST DATA
stocks = {
    "S1": {"width": 1219, "weight": 4395 },
    "S2": {"width": 1219, "weight": 9260},
    "S3": {"width": 1018, "weight": 3475},
    "S4": {"width": 1219, "weight": 8535},
    "S5": {"width": 236, "weight": 1571},
}

finish = {
    "F1": {"width": 235, "need_cut": 800 },
    "F2": {"width": 147, "need_cut": 1308},
    "F3": {"width": 136, "need_cut": 1290},
    "F4": {"width": 68, "need_cut": 600},
    "F5": {"width": 60, "need_cut": 170},
    "F6": {"width": 85, "need_cut": 132},
    "F7": {"width": 57, "need_cut": 100},
    "F8": {"width": 92, "need_cut": 100}, 
    "F9": {"width": 57, "need_cut": 735}, 
}

## II. Patterns

One approach to solving this problem is to create a list of all finished parts, a list of stocks for each length, and then use a set of binary decision variables to assign each finished product to a particular piece of stock. This approach will work well for a small problems, but the computational complexity scales much too rapidly with the size of the problem to be practical for business applications.

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to fit the width of the Mother Coil and under the weight demand of the Finished Goods

**A pattern $p$** is specified by the stock $s_p$ assigned to the pattern and 

integers $ap_{f}$ that specify the maximum of the finished parts of type $f$ are cut from stock $s_p$ and 
$\text{margin}^S_{s}$ is the allowed minimum margin of the according steel coil type; 
$wu^S_{s}$ the weight unit per mm wight of the coil stock, which is the weight divided by the MC width; 
$d^F_f$ the weight demand of the Finished Goods. A pattern $p\in P$ is feasible if <br>

$$
\begin{align}
& ap_{f}w^F_f  \leq   w^S_{s} - \text{margin}^S_{s} && \forall f\in F\\
& wu_{s} \times ( ap_{f} w^F_f) \leq d^F_f && \forall f\in F
\end{align}
$$

The function `make_patterns` defined below produces a partial list of feasible *naive* patterns for given sets of stocks and finished parts. Each pattern is represented as dictionary that specifies an associated stock item, and a dictionary of cuts that specify the finished parts cut from the stock. The algorithm is simple, it just considers every finished parts and stock items, then reports the number of parts $f$ that can be cut from stock item $s$.

### 2.1 Make Pattern Signature

The function signature of `make_patterns_by_weight_width` is as follows:

```python
from codes.create_patterns import make_patterns_by_weight_width

In [ ]:
from codes.create_patterns import make_patterns_by_weight_width
print(make_patterns_by_weight_width.__doc__)

In [ ]:
naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND, MIN_MARGIN)
len(naive_patterns)

Here, we will assume there exists the optimal solution in the combination created from the naive pattern


### 2.2 Cut Pattern Signature

The function signature of `generate_cut_combinations` is as follows:

```python
from codes.create_patterns import generate_cut_combinations


In [ ]:
from codes.create_patterns import ap_stock_bound, generate_cut_combinations

print(generate_cut_combinations.__doc__)

In [ ]:
from codes.create_patterns import ap_stock_bound, generate_cut_combinations

patterns = []
for s in stocks:
    max_cuts_dict, min_cuts_dict = ap_stock_bound(naive_patterns,finish,s)
    generate_cut_combinations(s, min_cuts_dict, max_cuts_dict, patterns)


## III. Optimal cutting using known patterns
Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to meet the demand for finished parts at minimum lost.

Let binary variable $b_{p}\in\mathbb{Z}_2$ denote the assignment of chosen stock to pattern $p$,

\begin{align}
\min \text{trim cost}: \quad &  w^S_s - \sum_{p\in P} \sum_{f\in F}b_{p} a_{fp}w^F_f \\
\text{s.t.} \quad
& \sum_{p\in P}b_{p} =  1 \\
& 96\% \times  w^S_s \leq  \sum_{p\in P} \sum_{f\in F}b_{p} a_{fp}w^F_f \leq  w^S_s -\text{margin}_{s} \\
& \sum_{p \in P}b_{p} wu^S_{s} \times (a_{fp} w^F_{f})  \leq d^F_f && \forall f\in F \\

& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P , \\
& b_{p} \in \{0,1\} &&  \forall p\in P \\
\end{align}

### 3.1.Explaination:
##### A. Pattern Constraint
For each *Pattern*, find one stock that fit the requirement of minimization
$$
\sum_{P\in P}b_{p} =  1 \quad 
$$
##### B. Width Demaind Constraint
The width of the finished goods cut for each item $f \in F$ 
should be at least 96% of the width of the mother coil (with a maximum 4% trim loss), but no greater than the width of the mother coil minus the stock margin."
$$
0.96 \times w^S_s \leq \sum_{p\in P} \sum_{f\in F}  b_{p} a_{fp}w^F_f \leq  w^S_s -\text{margin}_{s} \quad \forall p\in P 
$$

##### C. Weight Demand Constraint
Find, for each *Pattern*, the stock whose weight ensures that the combined weight of the after-cut pieces is either less than or equal to the upper-bound weight of the Finished Goods.
$$
\sum_{p\in P} b_{p} wu^S_{s} \times (a_{fp} w^F_{f}) \leq d_f \quad \forall f \in F 
$$

### 3.2 Cut Pattern Signature

The function signature of `cut_patterns_by_stock` is as follows:

```python
from codes.optima_sol import cut_patterns_by_stock
"""
        set F;
        set P;

        # width stock
        param width_s integer;
        # weight per unit of stock
        param wu > 0;
        # width finished pieces
        param width_f{F};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        # how many f pieces are returned from pattern p
        param a{F, P};
        # which stock s is choosen for pattern p
        var b{P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss:
          width_s - sum{p in P, f in F} b[p] * a[f,p] * width_f[f];
        
        subject to assign_each_finish_to_pattern:
          sum{p in P} b[p]  = 1;
        
        subject to feasible_pattern_max_margin:
          sum{p in P, f in F} b[p]* a[f,p] * width_f[f] >= 0.96 * width_s;
        
        subject to feasible_pattern_min_margin:
          sum{p in P, f in F}  b[p] * a[f,p] * width_f[f] <= width_s - 8;

        subject to weight_demand {f in F}:
          sum{p in P} b[p] * a[f,p] * width_f[f] * wu <= f_upper_demand[f];
    """

In [ ]:
from codes.optima_sol import cut_patterns_by_stock

for s in stocks.keys():
    filtered_patterns = [item for item in patterns if item['stock'] == s]
    opt_patterns = cut_patterns_by_stock(
                                     stocks[s]["width"]
                                      ,stocks[s]["weight"] 
                                     ,finish 
                                     ,filtered_patterns
                                     )

    if len(opt_patterns) == 0:
        print(f"No optimal solution with stock {s}")
        print("=================================")
    
    else:
        print(f"Solution for stock {s}")
        for p in opt_patterns:
            print(f"take pattern {p}")
            print("*****")
            cuts_dict= filtered_patterns[p]['cuts']
            print(cuts_dict)
            trim_loss = stocks[s]["width"] - sum([finish[f]["width"]*cuts_dict[f] for f in finish.keys()])
            weight_loss = trim_loss * stocks[s]["weight"]/stocks[s]["width"]
            print(f"trim loss: {trim_loss}, weight loss: {weight_loss}")
        print("=================================")